In [ ]:
from snappy import ProductIO
import snappy
import jpy

def LatLon_from_XY(product, x, y):
    geoPosType = jpy.get_type('org.esa.snap.core.datamodel.GeoPos')
    geocoding = product.getSceneGeoCoding()
    geo_pos = geocoding.getGeoPos(snappy.PixelPos(x, y), geoPosType())
    if str(geo_pos.lat)=='nan':
        raise ValueError('x, y pixel coordinates not in this product')
    else:
        return geo_pos.lat, geo_pos.lon

product = ProductIO.readProduct('ShipTest/S1A_IW_GRDH_1SDV_20200718T171436_20200718T171501_033512_03E220_DEFC_msk.tif')
print(LatLon_from_XY(product,0,0))

In [ ]:
import pandas as pd
detections = pd.read_csv('ShipTest/ResultsDetection/ShipsDetected_0.1.csv')

In [ ]:
LAT, LON = [], []
for ship in detections.iloc:
    lat, lon = LatLon_from_XY(product, ship.x, ship.y)
    LAT.append(lat)
    LON.append(lon)

In [ ]:
ship_det_LL = pd.DataFrame({'latitude':LAT,'longitude':LON})
ship_det_LL.to_csv('ship_det_LL.csv') 

# Create SHP

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon

style = 'symbol:pin; fill:#0000ff; fill-opacity:0.7; stroke:#ffffff; stroke-opacity:1.0; stroke-width:0.5'
text = None
dateTime = None

In [2]:
df = pd.read_csv('ship_det_LL.csv')
df

,Unnamed: 0,latitude,longitude
0,0,44.420569,8.733171
1,1,44.425762,8.753170
2,2,44.425306,8.784408
3,3,44.412296,8.706850
4,4,44.421193,8.797485
...,...,...,...
214,214,42.983903,9.342531
215,215,43.045706,9.847175
216,216,43.043226,9.798806
217,217,43.038506,9.798189


In [4]:
SHP = gpd.GeoDataFrame(columns = ['style_css', 'label', 'text', 'dateTime', 'geometry'])

for i in range(len(df)):

    label = f'Ship Detected {i}'
    lat, lon = df['latitude'][i], df['longitude'][i]
    coordinates = [(lon, lat)]
    geometry = Point(coordinates)

    new_row = {'style_css':style, 'label':label, 'text':text, 'dateTime':dateTime, 'geometry':geometry   }
    SHP = SHP.append(new_row, ignore_index=True)
    
SHP

,style_css,label,text,dateTime,geometry
0,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 0,None,None,POINT (8.73317 44.42057)
1,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 1,None,None,POINT (8.75317 44.42576)
2,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 2,None,None,POINT (8.78441 44.42531)
3,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 3,None,None,POINT (8.70685 44.41230)
4,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 4,None,None,POINT (8.79748 44.42119)
...,...,...,...,...,...
214,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 214,None,None,POINT (9.34253 42.98390)
215,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 215,None,None,POINT (9.84717 43.04571)
216,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 216,None,None,POINT (9.79881 43.04323)
217,symbol:pin; fill:#0000ff; fill-opacity:0.7; st...,Ship Detected 217,None,None,POINT (9.79819 43.03851)


In [5]:
!mkdir esri 

In [6]:
SHP.to_file('esri/navi.shp')